# Practicing link prediction with PyG

## Using a 'custom' `decoder` function
base: https://github.com/pyg-team/pytorch_geometric/blob/master/examples/link_pred.py

In [46]:
import torch

import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from torch_geometric.utils import negative_sampling
from torch_geometric.data import Data
# from torch_geometric.utils import train_test_split_edges

from sklearn.metrics import roc_auc_score

In [4]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')


In [19]:
transform = T.Compose([ # composite transform
    T.NormalizeFeatures(),
    T.ToDevice(device),
    T.RandomLinkSplit(num_val=0.05, num_test=0.1) # 5% for validation, 10% for testing and the rest 85% for training
])

In [16]:
dataset = Planetoid(root='datasets/Cora', name='Cora', transform=transform)
dataset

Cora()

In [37]:
dataset_not_transformed = Planetoid(root='datasets/Cora', name='Cora')
dataset_not_transformed[0]

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [40]:
dataset[0] # what are the `edge_label` and `edge_label_index`
# i think, the `RandomLinkSplit` adds one negative samples for every positive samples in each of the returning graphs

(Data(x=[2708, 1433], edge_index=[2, 8974], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], edge_label=[17948], edge_label_index=[2, 17948]),
 Data(x=[2708, 1433], edge_index=[2, 8974], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], edge_label=[1054], edge_label_index=[2, 1054]),
 Data(x=[2708, 1433], edge_index=[2, 9501], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], edge_label=[2110], edge_label_index=[2, 2110]))

In [18]:
train_data, val_data, test_data = dataset[0]
train_data

Data(x=[2708, 1433], edge_index=[2, 8974], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], edge_label=[17948], edge_label_index=[2, 17948])

In [42]:
train_data.edge_index == val_data.edge_index

tensor([[True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True]])

In [28]:
train_data.is_undirected()

False

In [38]:
train_data.edge_label.unique(return_counts=True)

(tensor([0., 1.]), tensor([8974, 8974]))

In [34]:
train_data.edge_label_index

tensor([[1867,  201,  598,  ...,  727,  722, 1399],
        [2117,  598, 1003,  ...,  608, 1967,  977]])

In [35]:
train_data.edge_index

tensor([[1867,  201,  598,  ..., 2448, 1013, 1359],
        [2117,  598, 1003,  ..., 1067, 1661, 2480]])

In [21]:
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)
    
    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)
    
    def decode(self, z, edge_label_index):
        return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(dim=1) # sum(subject_nodes * object_nodes)
    
    def decode_all(self, z):
        prob_adj = z @ z.t()
        return (prob_adj > 0).nonzero(as_tuple=False).t()
    
    
    def forward(self, data: Data): 
        z = self.encode(data.x, data.edge_index)
        
        if not self.training:
            return self.decode(z, data.edge_label_index).view(-1)

        # calculating a new set of negative samples
        neg_edge_index = negative_sampling(
            edge_index = train_data.edge_index,
            num_nodes = train_data.num_nodes,
            num_neg_samples = train_data.edge_label_index.size(1),
            method='sparse'
        )

        # adding those negative sample to the data
        edge_label_index = torch.cat([
            train_data.edge_label_index, 
            neg_edge_index
        ], dim=-1)

        edge_label = torch.cat([
            train_data.edge_label,
            train_data.edge_label.new_zeros(neg_edge_index.size(1))
        ], dim = 0)

        return self.decode(z, edge_label_index).view(-1)


        

In [22]:
model = GCN(dataset.num_features, 128, dataset.num_classes).to(device)
model

GCN(
  (conv1): GCNConv(1433, 128)
  (conv2): GCNConv(128, 7)
)

In [43]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()

In [30]:
train_data.edge_label_index.shape

torch.Size([2, 17948])

In [51]:
def train():
    model.train()
    optimizer.zero_grad()

    z = model.encode(train_data.x, train_data.edge_index)

    # calculating a new round of negative sampling for every every epoch
    neg_edge_index = negative_sampling(
        edge_index = train_data.edge_index,
        num_nodes = train_data.num_nodes,
        num_neg_samples = train_data.edge_label_index.size(1),
        method='sparse'
    )

    # adding concatenating the new neg sample with the existing samples
    edge_label_index = torch.cat([
        train_data.edge_label_index, 
        neg_edge_index
    ], dim=-1)


    edge_label = torch.cat([
        train_data.edge_label,
        train_data.edge_label.new_zeros(neg_edge_index.size(1))
    ], dim = 0)

    out = model.decode(z, edge_label_index).view(-1)
    
    loss = criterion(out, edge_label)
    loss.backward()
    optimizer.step()
    return loss
    

the codes bellow this, are direct copy-paste, without reading anything.

In [54]:
@torch.no_grad()
def test(data: Data):
    model.eval()
    z = model.encode(data.x, data.edge_index)
    out = model.decode(z, data.edge_label_index).view(-1).sigmoid()
    return roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())
    

In [55]:
best_val_auc = final_test_auc = 0
for epoch in range(1, 101):
    loss = train()
    val_auc = test(val_data)
    test_auc = test(test_data)
    if val_auc > best_val_auc:
        best_val_auc = val_auc
        final_test_auc = test_auc
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val: {val_auc:.4f}, '
          f'Test: {test_auc:.4f}')

print(f'Final Test: {final_test_auc:.4f}')

z = model.encode(test_data.x, test_data.edge_index)
final_edge_index = model.decode_all(z)

Epoch: 001, Loss: 0.6302, Val: 0.8624, Test: 0.8547
Epoch: 002, Loss: 0.6274, Val: 0.8647, Test: 0.8586
Epoch: 003, Loss: 0.6283, Val: 0.8661, Test: 0.8607
Epoch: 004, Loss: 0.6278, Val: 0.8675, Test: 0.8616
Epoch: 005, Loss: 0.6270, Val: 0.8681, Test: 0.8632
Epoch: 006, Loss: 0.6254, Val: 0.8688, Test: 0.8651
